# Scraping Metacritic

## Dexter

Allons récupérer les notes des utilisateurs pour Dexter sur Metacritic (https://www.metacritic.com/tv/dexter/user-reviews).

Pour l'instant, on récupère juste la première page pour ne pas risquer le <font color='red'> **ban** </font>.

C'est par saison et pas par série, je suis la tristesse.

In [7]:
import requests
from bs4 import BeautifulSoup

In [5]:
url = "https://www.metacritic.com/tv/dexter/user-reviews"
# http://jacketheredge.com/Webscraping+Metacritic.html - pour éviter 403 (forbidden)
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 

response = requests.get(url, headers=headers)
response

<Response [200]>

### Récupérer la première review (nom utilisateur + note)

In [8]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [21]:
review = html_soup.find('div', class_ = 'review_section')

In [25]:
user = review.a.text

In [26]:
user

'NYcritic'

In [35]:
r = review.find('div', class_="review_grade")

In [37]:
rating = int(r.div.text)

In [38]:
rating

9

### Récupérer toutes les reviews de la page


In [56]:
nb_reviews = 100 # nb of reviews per page
reviews = html_soup.find_all('div', class_ = 'review_stats')
len(reviews)

103

In [65]:
r = reviews[1]
r

<div class="review_stats">
<div class="review_critic">
<div class="name">
<span>rossc</span>
</div>
<div class="date">Oct 17, 2006</div>
</div>
<div class="review_grade">
<div class="metascore_w user medium tvshow positive indiv perfect">10</div>
</div>
</div>

In [69]:
n = r.find('div', class_='name')

In [72]:
list(n.children)[1].text

'rossc'

In [74]:
r0 = reviews[0]
n0 = r0.find('div', class_='name')
list(n0.children)[1].text

'NYcritic'

In [77]:
d = dict()
for i in range(0, nb_reviews) :
    name = reviews[i].find('div', class_='name')
    user = list(name.children)[1].text
    r = reviews[i].find('div', class_="review_grade")
    rating = int(r.div.text)
    d[user] = {'Dexter' : rating}


Ça marche bien mais plusieurs problèmes :
- *notes par saison (POURQUOI :'( )*
    - on pourrait faire une moyenne des notes sur chaque saison pour un utilisateur 
    - tous les utilisateurs qui ont noté la saison 1 de Dexter n'ont pas noté la saison 2 (et 3 et 4, etc) de Dexter
    - on pourrait imaginer une pénalité dans la moyenne pour chaque saison non notée (on la considère comme non regardée)
    
    
- *comment faire pour récupérer depuis les noms des dossiers de sous-titres ?*
    - pour Dexter, pas de soucis, on met en minuscule et on est bon
    - pour CSI par exemple, le nom dans le dossier des sous-titres est "CSI_Crime_Scene_Invertigation" mais c'est seulement "csi" dans l'url du site
    
    
- *les noms de séries en plusieurs mots mais sans acronymes ?? (apparemment avec des '-' dans l'url mais pas forcément les mêmes noms)*
    - possibilité de récupérer les noms que le site utilise ? mais comment les relier avec nos titres après